See problem description for [day 5 part 2](https://adventofcode.com/2022/day/5#part2)

In [1]:
# STRIPPING WHITESPACE ON THE LEFT is NOT USEFUL in this problem.
def get_lines(file_type='sample'):
    '''
    Read in the lines of today's sample/input file line by line. 
    Assumes the file is in folder called 'inputs/'
    
    Parameters
    ----------
    file_type : str
        Either sample or input
    
    Returns
    -------
    list of inputs stripped of whitespace
    '''
    import datetime
    day = str(datetime.datetime.today().day).zfill(2)
    filename = f'inputs/{day}-{file_type}.txt'
    try:
        with open(filename,'r') as file:
            lines = [line.rstrip() for line in file.readlines()]
        return lines
    except:
        print(filename+' does not exist')

In [3]:
sample_config = get_lines()

In [4]:
sample_config

['    [D]',
 '[N] [C]',
 '[Z] [M] [P]',
 ' 1   2   3',
 '',
 'move 1 from 2 to 1',
 'move 3 from 1 to 3',
 'move 2 from 2 to 1',
 'move 1 from 1 to 2']

In [5]:
input_config = get_lines('input')

In [7]:
def get_number_of_stacks(config):
    for i, line in enumerate(config):
        for ch in line:
            if ch.isdigit():
                return (i, int(line[-1]))
    return -1

In [9]:
s_labels, s_stacks = get_number_of_stacks(sample_config)
s_labels, s_stacks

(3, 3)

In [10]:
i_labels, i_stacks = get_number_of_stacks(input_config)
i_labels, i_stacks

(8, 9)

In [15]:
def get_value(line, stack):
    try:
        ch = line[1+(stack-1)*4]
        if ch.isalpha():
#             print(ch)
            return ch
        else:
#             print('no value: before last valid')
            return None
    except:
#         print('no value: after last valid')
        return None

In [16]:
from queue import LifoQueue

In [17]:
def initialize_state(config):
    labels, num_stacks = get_number_of_stacks(config)
    stacks = {i: LifoQueue() for i in range(1,num_stacks+1)}
    for i in range(labels-1,-1,-1):
        for stack in range(1, num_stacks+1):
            crate = get_value(config[i], stack)
#             print(crate)
            if crate is not None:
                stacks[stack].put(crate)
#         print('\n')
    return stacks

### Initialize the state of the stacks

In [18]:
s_stacks = initialize_state(sample_config)

In [19]:
i_stacks = initialize_state(input_config)

### Get the moves
These start 2 below the labels.

In [21]:
s_moves = sample_config[s_labels+2:]
i_moves = input_config[i_labels+2:]

In [24]:
def extract_moves(move):
    extract = move.split(' ')
    num_to_move = int(extract[1])
    from_stack = int(extract[3])
    to_stack = int(extract[5])
    return (num_to_move, from_stack, to_stack)

In [26]:
s_moves

['move 1 from 2 to 1',
 'move 3 from 1 to 3',
 'move 2 from 2 to 1',
 'move 1 from 1 to 2']

In [25]:
for move in s_moves:
    print(extract_moves(move))

(1, 2, 1)
(3, 1, 3)
(2, 2, 1)
(1, 1, 2)


### Helpful functions to check work:
Check the stack size and identify the top crate on each stack

In [27]:
def check_stack_sizes(stacks):
    for stack in stacks:
        print(stack, stacks[stack].qsize())

In [34]:
def get_top_crates(stacks):
    '''
    Leave the state of the stacks alone! 
    '''
    answer = ''
    for stack in stacks:
        ch = stacks[stack].get()
        answer += ch
        stacks[stack].put(ch)
    return answer

### Define how to move a substack, then make the moves!

In [28]:
def move_substack(stacks, move):
    num_to_move, from_stack, to_stack = extract_moves(move)
    temp_stack = LifoQueue()
    for n in range(num_to_move):
        crate = stacks[from_stack].get()
        temp_stack.put(crate)
    for n in range(num_to_move):
        crate = temp_stack.get()
        stacks[to_stack].put(crate)
    return stacks

In [31]:
def make_moves(stacks, moves):
    for move in moves:
        stacks = move_substack(stacks, move)
    return stacks

In [32]:
s_stacks = make_moves(s_stacks, s_moves)
check_stack_sizes(s_stacks)

1 1
2 1
3 4


In [33]:
i_stacks = make_moves(i_stacks, i_moves)
check_stack_sizes(i_stacks)

1 8
2 5
3 9
4 5
5 15
6 5
7 1
8 3
9 5


### Get the answer...
Fingers crossed 🤞

In [35]:
get_top_crates(s_stacks)

'MCD'

In [36]:
get_top_crates(i_stacks)

'DCVTCVPCL'

### My skills are improving.
I've gotten all 10 stars on the first submission (so far).